In [2]:
import sys

sys.path.append('/Users/kallewesterling/Dropbox (Graduate Center)/dev-dhri/django-app/')

In [81]:
from pathlib import Path
import json

data = json.loads(Path('../../lessons.json').read_text())

In [ ]:
for i, heading in enumerate(data):
    if heading.startswith("# "):
        print(i)
        print(f"------ STARTING HEADER 1: {heading} ————————")
        print(data[heading])
        x = (item for item in data.items())
        done, in_codeblock = False, False
        ii = -1
        while done == False:
            ii += 1
            if ii <= i:
                next(x)
                continue
            
            # We are now moving ahead... get the next line
            try:
                next_heading, next_body = next(x)
                print("NEXT")
            except StopIteration:
                done = True
            
            is_codestarter = next_body[:3] == '```'
            if is_codestarter:
                if in_codeblock == False:
                    in_codeblock = True
                    print('in codeblock')
                elif in_codeblock == True:
                    in_codeblock = False
                    print('no longer')
                
            if next_heading.startswith("# ") and in_codeblock == False:
                done = True
            elif next_heading.startswith("# ") and in_codeblock == True:
                print(next_heading)
                print(next_body)
            else:
                print(next_heading)
                print(next_body)

In [29]:
from pathlib import Path
from collections import OrderedDict

In [48]:


data = Path('../../lessons.txt').read_text()

ordered_data = OrderedDict()

all_lines = [x for x in data.splitlines() if x]

for i, line in enumerate(all_lines):
    if line.startswith("# "):
        x = (item for item in all_lines)
        ii, done, in_codeblock = 0, False, False
        ordered_data[line] = OrderedDict()
        
        while done == False:
            skip = False
            
            if i >= ii:
                next(x)
                ii += 1
                skip = True

            if skip:
                continue
                
            try:
                nextline = next(x)
            except StopIteration:
                done = True
            
            is_codestarter = nextline[:3] == '```'
            is_new_header = nextline[:2] == '# '
            
            ordered_data[line][ii] = {}
            
            if is_codestarter:
                if in_codeblock == True:
                    print('done')
                    in_codeblock = False
                    ii += 1
                    skip = True
                elif in_codeblock == False:
                    print('in codeblock')
                    language = nextline[3:].strip()
                    print(language)
                    in_codeblock = True
                    ii += 1
                    skip = True
            
            if skip: continue
                
            if in_codeblock:
                print(nextline)
                ii += 1
                skip = True
            
            if skip: continue
                
            print(nextline)
            
            ii += 1

## What is Git? 
**Git** is software used for version control—that is, tracking the state of files and changes you make to them over time. Git can be enabled in a folder, and then used to save the state of the contents in that folder at different points in the future, as designated by you. In the language of Git, a folder is called a *repository*. In the context of this workshop, it refers to a folder that is being tracked by Git. Using Git, you can view a log of the changes you've made to the files in a repository and compare changes over time. You can also revert back to previous versions, and create branches of a project to explore different futures. Git is also useful for collaboration, as a repository can be shared across computers, and its contents can be asynchonously developed and eventually merged with the main project. 
## What is GitHub?
**GitHub** is a online platform for hosting Git repositories. It functions for some, predominantly programmers, as a social network for sha

	
If the command is successful, the full Git lesson will be replicated on your local machine. You can type
	cd git
	
to enter the lesson folder, since the lesson repository is simply called `git`. Use the `ls` command to take a look at the various files in the lesson folder.
Cloning can be especially useful when you're joining a group project that is hosted on GitHub, and you want your changes to eventually be pushed and shared with that same repository. 
Forking is a step you migth take before closing, if you want to establish a separate home for the future changes you make to the repository on your GitHub account rather than push to an existing folder owned by someone else on GitHub. 
## Forking
**Forking** a repository means making a copy of someone else's repository on GitHub, and saving it to your account on GitHub. This function happens within GitHub, and has nothing to do with what is happening on your local machine. 
For example, go to [the repository for this workshop](https:/